In [1]:
!pip install sentence_transformers dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.6 MB/s eta 0:00:0000:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.35
    Uninstalling SQLAlchemy-2.0.35:
      Successfully uninstalled SQLAlchemy-2.0.35
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, CrossEncoder, util,SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
import pandas as pd
import numpy as np
import torch
from datasets import Dataset

# Bước 1: Đọc dataset NLI và QA sau đó tách content ra làm file Corpus

In [4]:
Corpus = pd.read_csv("/kaggle/input/healthcare-chatbot/corpus_healthcare.csv")
train = pd.read_csv("/kaggle/input/healthcare-chatbot/train_healthcare.csv")
test = pd.read_csv("/kaggle/input/healthcare-chatbot/test_healthcare.csv")
val = pd.read_csv("/kaggle/input/healthcare-chatbot/val_healthcare.csv")

***corpus dài nhất có: 4437 token***

# BI-ENCODER FINETUNE

In [4]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import pandas as pd
from sentence_transformers import util

In [5]:
from sentence_transformers import SentenceTransformer
import torch

# Kiểm tra xem GPU có sẵn không
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Tải mô hình và chỉ định thiết bị
model_path = "/kaggle/input/halong_embedding_4jan/other/default/1/checkpoint-3500"
model = SentenceTransformer(model_path, device=device)

# Kiểm tra xem model đã được tải trên GPU hay không
print(f"Model device: {model.device}")


Using device: cuda
Model device: cuda:0


In [6]:
from sentence_transformers import CrossEncoder
import torch

# Kiểm tra xem GPU có sẵn không
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Tải mô hình và chỉ định thiết bị
model_name = "/kaggle/input/bge_finetune_6file/transformers/default/1/bge_reranker_finetune_24Dec_latest5"
cross_encoder = CrossEncoder(model_name, num_labels=1, max_length=1024, device=device)


Using device: cuda


In [7]:
import logging
logging.disable(logging.WARNING)

In [8]:

model_cross = cross_encoder.model

# Print the model architecture
print(model_cross)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_f

In [9]:
corpus = pd.read_csv("/kaggle/input/healthcare-chatbot/corpus_healthcare.csv")

In [10]:

import torch

# Đường dẫn đến tệp .pt
file_path = "/kaggle/input/embedding-doc/doc_embeddings_hl_ft.pt"

# Tải dữ liệu
data = torch.load(file_path)

# Kiểm tra và xử lý dữ liệu
if isinstance(data, torch.Tensor):
    # Nếu toàn bộ file là một tensor
    embeddings = data
elif isinstance(data, dict) and 'embeddings' in data:
    # Nếu file là dictionary và chứa key 'embeddings'
    embeddings = data['embeddings']
else:
    raise ValueError("File không chứa embeddings hoặc không đúng định dạng!")

# Hiển thị thông tin embeddings
print(f"Tổng số embeddings: {len(embeddings)}")
print("Embedding mẫu:", embeddings[0])

# Đảm bảo kiểu dữ liệu là tensor
if isinstance(embeddings, torch.Tensor):
    print(f"Kích thước của embedding: {embeddings.shape}")


<ipython-input-10-01ffa0a314a2>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


Tổng số embeddings: 201278
Embedding mẫu: tensor([ 2.4059e-02, -3.3965e-02,  2.7742e-02, -1.0075e-02,  4.8988e-02,
        -1.9550e-02,  2.8276e-02,  6.6316e-03, -7.6046e-02, -3.5337e-02,
        -2.5771e-02,  2.9420e-03, -3.3403e-02,  3.5558e-02,  2.1970e-02,
         6.5235e-02,  3.8740e-02, -2.9432e-02,  1.5462e-02, -4.5378e-02,
        -1.7632e-02, -4.9764e-02,  7.2799e-03, -1.3141e-02,  4.5410e-02,
         6.4563e-03, -1.1320e-02, -6.8925e-02,  3.3045e-02,  3.9826e-02,
        -4.1069e-02,  5.9556e-02, -4.6521e-02,  1.0189e-02,  3.5060e-02,
        -9.0544e-04,  1.6474e-02,  5.6453e-02, -3.8884e-02,  1.8709e-02,
         1.6357e-02, -4.0798e-04,  3.1813e-03,  5.2386e-03,  2.1488e-02,
         2.4168e-02,  3.5207e-03,  6.2477e-02, -4.4913e-02,  6.8346e-03,
         7.1183e-02, -4.1380e-02, -2.4523e-02, -1.8445e-02, -3.5180e-02,
        -2.2221e-02,  4.8468e-02,  1.6551e-03,  3.7592e-02, -1.0580e-03,
        -7.6505e-02,  2.0165e-02,  1.5811e-02, -1.8044e-02,  3.5696e-02,
         

In [11]:
import re

def preprocess_text(text):
    # Replace newline characters with spaces
    text = ' '.join(text.splitlines())
    
    # Replace escaped newline characters with periods
    text = text.replace('\\n', '.')
    
    # Remove specific unwanted characters and patterns
    text = text.replace('/', '')
    text = text.replace("''", '')
    text = text.strip("[]")
    text = text.strip("'")
    text = text.strip('"')
    
    # Remove numbered list indicators and lettered list indicators
    text = re.sub(r'\b\d\.\s*', '', text)  # Removes patterns like '1. ', '2. ', etc.
    text = re.sub(r'\b[a-e]\)\s*', '', text)  # Removes patterns like 'a) ', 'b) ', etc.
    
    # Replace certain punctuation with periods
    text = text.replace(';', '.')
    text = text.replace(':', '.')
    
    # Remove hyphens
    text = text.replace('-', '')
    
    # Strip leading and trailing whitespace
    text = text.strip()

    text = re.sub(r'\s{2,}', ' ', text)

    
    return text



In [ ]:
    torch.cuda.empty_cache()
    
    # Query đầu vào
    query = "Sốt xuất huyết ở trẻ em nguy hiểm thế nào khi không xử lý đúng cách?"
    
    # Encode query bằng model
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Tìm top-k văn bản liên quan nhất trong embeddings
    top_k = 50
    hits = util.semantic_search(query_embedding, embeddings, top_k=top_k)[0]
    
    # Hiển thị thông tin query
    print(query)
    print("")
    print("Context:")
    print("")
    
    # In top 10 kết quả dựa trên điểm bi-encoder (bi_score)
    for hit in hits[:10]:
        score = hit['score']
        corpus_id = hit['corpus_id']
        print("bi_score: ", score)
        print(corpus.iloc[corpus_id]['context'])  # Truy cập cột context trong corpus
        print(" ")
    
    # Chuẩn bị input cho cross-encoder
    # preprocess_text là hàm tiền xử lý nếu cần, nếu không cần thì giữ nguyên văn bản
    cross_inp = [[query, corpus.iloc[hit['corpus_id']]['context']] for hit in hits]
    
    # Tính điểm cross-encoder
    cross_scores = cross_encoder.predict(cross_inp)
    
    # Cập nhật điểm cross-score cho các hits
    for idx in range(len(hits)):
        hits[idx]['cross-score'] = cross_scores[idx]
    
    # Sắp xếp hits theo cross-score (giảm dần)
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    
    # In top 10 kết quả dựa trên điểm cross-encoder (cross_score)
    for hit in hits[:10]:
        score = hit['cross-score']
        corpus_id = hit['corpus_id']
        print("cross_score: ", score)
        print(corpus.iloc[corpus_id]['context'])  # Truy cập cột context trong corpus
        print(" ")
    
    # Giải phóng bộ nhớ GPU (nếu cần)
    torch.cuda.empty_cache()

In [14]:
import numpy as np
import torch
import pandas as pd
from sentence_transformers import util

def calculate_mrr(query_file, corpus_file, doc_embedding_file, model, cross_encoder, top_k=50):
    """
    Tính Mean Reciprocal Rank (MRR) khi kết hợp cả bi-encoder và cross-encoder.

    Parameters:
    - query_file: Đường dẫn tới file chứa các truy vấn.
    - corpus_file: Đường dẫn tới file chứa corpus.
    - doc_embedding_file: Đường dẫn tới file chứa document embeddings.
    - model: Bi-encoder model dùng để tính embeddings.
    - cross_encoder: Cross-encoder model dùng để tính điểm cho các cặp (query, document).
    - top_k: Số lượng văn bản được chọn từ bi-encoder để đưa vào cross-encoder.

    Returns:
    - mrr: Giá trị Mean Reciprocal Rank (float).
    """
    reciprocal_ranks = []

    # Load queries và corpus
    query_df = pd.read_csv(query_file).head(100)  # Chỉ lấy 1000 query đầu tiên
    corpus_df = pd.read_csv(corpus_file)

    queries = query_df['query'].tolist()
    corpus = corpus_df['context'].tolist()

    # Load document embeddings
    doc_embeddings = torch.load(doc_embedding_file)
    doc_embeddings = doc_embeddings.cpu().numpy() if doc_embeddings.is_cuda else doc_embeddings.numpy()

    for query_idx, query in enumerate(queries):
        # Encode query bằng model
        query_embedding = model.encode(query, convert_to_tensor=True)
        query_embedding = query_embedding.cpu().numpy() if query_embedding.is_cuda else query_embedding.numpy()

        # Bi-encoder phase: Tính điểm similarity và lấy top-k văn bản liên quan nhất
        scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)
        scores = scores.squeeze().numpy()
        top_k_indices = np.argsort(scores)[::-1][:top_k]

        # Chuẩn bị input cho cross-encoder
        cross_inp = [[query, corpus[idx]] for idx in top_k_indices]

        # Cross-encoder phase: Tính điểm
        cross_scores = cross_encoder.predict(cross_inp)

        # Sắp xếp lại theo điểm cross-score
        ranked_indices = [top_k_indices[idx] for idx in np.argsort(cross_scores)[::-1]]

        # Tìm vị trí của câu trả lời đúng
        relevant_idx = query_idx  # Đồng bộ query và corpus
        rank_position = next((idx + 1 for idx, doc_idx in enumerate(ranked_indices) if doc_idx == relevant_idx), None)

        # Tính reciprocal rank
        if rank_position is not None:
            reciprocal_ranks.append(1 / rank_position)
        else:
            reciprocal_ranks.append(0)  # Nếu không tìm thấy câu trả lời đúng trong top-k

    # Tính Mean Reciprocal Rank
    mrr = np.mean(reciprocal_ranks)
    return mrr

# Đường dẫn tới file truy vấn, corpus và document embeddings
query_file = "/kaggle/input/testmrr/processed_queries.csv"
corpus_file = "/kaggle/input/testmrr/corpus_with_index.csv"
doc_embedding_file = "/kaggle/input/testmrr/doc_embeddings (1).pt"

# Giả sử model và cross_encoder đã được khởi tạo từ trước
# model = ...  # Bi-encoder model
# cross_encoder = ...  # Cross-encoder model

# Kiểm tra khởi tạo model và cross_encoder (cần thay thế bằng mô hình thực tế)
if 'model' not in locals() or 'cross_encoder' not in locals():
    raise ValueError("Cần khởi tạo model và cross_encoder trước khi chạy chương trình.")

# Chạy hàm calculate_mrr
mrr = calculate_mrr(query_file, corpus_file, doc_embedding_file, model, cross_encoder, top_k=50)
print(f"MRR: {mrr}")


<ipython-input-14-112b347480d8>:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  doc_embeddings = torch.load(doc_embedding_file)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

MRR: 0.07293627721341175


In [12]:
!pip install pyngrok
!ngrok config add-authtoken 2qORvsrf7DPupP5SxpxroXZcHrt_5wvaAdPmdAyqb7JHYYN21
!pip install flask-cors

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS
import requests  # Thư viện để gọi API HTTP

app = Flask(__name__)
CORS(app)

# URL server ngrok thay vì sử dụng hàm `phogpt`
NGROK_SERVER_URL = "https://f330-183-80-32-59.ngrok-free.app"  # Thay bằng URL ngrok của bạn

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Lấy dữ liệu từ request.form
        question = request.form.get('message')  # 'message' khớp với key trong JavaScript
        if not question:
            return jsonify({"error": "Missing question"}), 400

        query_embedding = model.encode(question)
        hits = util.semantic_search(query_embedding, embeddings, top_k=50)[0]
        cross_inp = [[preprocess_text(question), preprocess_text(corpus.iloc[hit['corpus_id']]['context'])] for hit in hits]
        cross_scores = cross_encoder.predict(cross_inp)
        for idx in range(len(hits)):
            hits[idx]['cross-score'] = cross_scores[idx]
        hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
        best_hit = hits[0]
        top_k_cross = corpus.iloc[best_hit['corpus_id']]['context']
        print('cross-encoder')
        
        # Cross-encoder refinement
        print(top_k_cross)
        # Gọi tới server ngrok thay vì dùng phogpt
        try:
            response = requests.post(
                f"{NGROK_SERVER_URL}/ask",  # Đường dẫn tới endpoint xử lý trên server ngrok
                json={
                    "question": question,
                    "context": top_k_cross  # Gửi thông tin cần thiết
                }
            )
            response_data = response.json()

            # Kiểm tra phản hồi từ server
            if response.status_code != 200 or "answer" not in response_data:
                return jsonify({"answer": "Error processing the request to the external server."})
            
            answer = response_data["answer"]
        except Exception as e:
            print(f"Error calling external server: {str(e)}")
            return jsonify({"answer": "Error contacting external server."})

        if not answer:
            return jsonify({"answer": "Sorry, I couldn't generate an appropriate response."})
        return jsonify({"answer": answer})

    except Exception as e:
        # Log lỗi và trả về phản hồi JSON lỗi
        print(f"Error: {str(e)}")
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500

# Start the Flask server
if __name__ == '__main__':
    public_url = ngrok.connect(5000)  # Kết nối Flask server với ngrok
    print(f"Public URL: {public_url}")
    app.run(host='0.0.0.0', port=5000)


Public URL: NgrokTunnel: "https://52e4-35-199-158-98.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
Đi tiểu là quá trình bình thường của cơ thể, mục đích chính là đào thải các chất dư thừa hay chất độc ra khỏi cơ thể. Nhiều người thắc mắc hay đi tiểu nhiều lần là bệnh gì? Để trả lời câu hỏi này chúng ta cần xác định số lần đi tiểu mỗi ngày của một người bình thường. Theo đó, vào ban ngày một người bình thường có thể đi tiểu khoảng 5 – 7 lần, còn về đêm tối đa chỉ đi 1 lần hoặc đa số là ngủ liên tục 8 tiếng mà không cần đi tiểu. Vì vậy, định nghĩa tiểu đêm nhiều lần là khi số lần đi tiểu mỗi đêm từ 2 lần trở lên, áp dụng cho cả nam và nữ giới Trong cuộc sống hằng ngày, không khó để bắt gặp tình trạng này và nam giới vẫn thắc mắc vào ban đêm đi tiểu nhiều lần là bệnh gì ? Đây đôi khi không phải là biểu hiện bệnh lý mà được xem là bình thường trong một số tình huống như: Tiểu đêm nhiều lần nhưng không xảy ra thường xuyên; Tiểu đêm nhiều lần do tiêu thụ thực phẩm chứa nhiều nước hoặc uống nước nhiều trước khi đi ngủ. Ngược lại, nếu đã loại trừ các yếu tố có thể gây tiểu đêm

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
Với chứng đau đầu do căng thẳng, bạn vẫn trông có vẻ ổn và bình thường dù đang chịu đựng các cơn đau, đồng thời không có các triệu chứng kèm theo khác. Bác sĩ có thể chẩn đoán đau đầu do căng thẳng qua mô tả của bệnh nhân. Ngoài một số cơn đau cơ xung quanh đầu, không có điểm gì bất thường để bác sĩ phát hiện và chẩn đoán phân biệt. Bạn cũng không cần thực hiện các xét nghiệm trừ khi có các triệu chứng bất thường hoặc nghi ngờ mắc phải tình trạng khác ngoài chứng đau đầu căng thẳng mãn tính.

Đặc biệt lưu ý, nên loại trừ chứng đau đầu do lạm dụng thuốc vì tình trạng này thường có thể nhầm với đau đầu căng thẳng mãn tính. Đau đầu do lạm dụng thuốc có thể tương tự như đau đầu căng thẳng mãn tính. Tình trạng này là do dùng thuốc giảm đau (hoặc thuốc triptan) quá thường xuyên, dẫn đến đau đầu do căng thẳng hoặc các cơn đau nửa đầu.

Đôi khi chứng **đau nửa đầu** cũng bị nhầm với đau đầu căng thẳng mãn tính. Một số người không thích đèn sáng hoặc tiếng ồn lớn, cũng như không m

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
Do đột quỵ không có triệu chứng báo hiệu kéo dài nên không ai có thể biết trước được khi nào mình sẽ bị đột quỵ. Vì vậy, bạn nên tham khảo, nắm vững những dấu hiệu cảnh báo sớm đột quỵ để phát hiện sớm tình trạng này và cấp cứu kịp thời: Dấu hiệu ở thị lực: Thị lực bị giảm, nhìn mờ 1 mắt hoặc cả 2 mắt. Tuy nhiên, đây là biểu hiện không rõ ràng nên khó nhận ra. Nếu người bệnh nhận thấy mình có dấu hiệu này thì nên yêu cầu được đi cấp cứu ngay; Dấu hiệu ở mặt: Mặt bệnh nhân có biểu hiện đột nhiên không cân xứng, nhân trung hơi lệch qua 1 bên, miệng méo, nếp mũi má bên yếu bị rũ xuống. Đặc biệt, khi bệnh nhân nói hoặc cười sẽ nhìn thấy rõ dấu hiệu méo miệng, thiếu cân xứng trên mặt; Dấu hiệu ở giọng nói: Dấu hiệu báo trước của tai biến đột quỵ có thể biểu hiện qua giọng nói. Người bệnh có thể bị nói ngọng bất thường, môi lưỡi tê cứng, khó mở miệng, gắng sức mới nói được; Dấu hiệu ở tay, chân: Người bệnh có thể cảm thấy tay bị tê mỏi, khó cử động và thao tác. Ngoài ra, bệnh n